# CC3001 Otoño 2020 Tarea 4
# Generación de código

Profesores: Sección 1 Patricio Poblete / Sección 2 Nelson Baloian

Plazo: Lunes 8 de junio de 2020 a las 23:59

El objetivo de esta tarea es aprender sobre la manipulación de árboles que representan fórmulas. Hay muchas cosas que uno puede hacer con estos árboles, y en esta tarea nos enfocaremos en particular en algo que los compiladores hacen: la **generación de código** para evaluar fórmulas.

Específicamente, el problema es: dada una fórmula, generar una secuencia de instrucciones elementales que la evalúen. Por ejemplo, dada una fórmula como

```
(a+b)*(c-d)
```

se podría generar código como el siguiente:

```
t1=a+b
t2=c-d
t3=t1*t2
```

Las instrucciones elementales son siempre de la forma ``ti=x op y``, donde ``ti`` es una variable temporal, ``x`` e ``y`` pueden ser variables o números, y ``op`` es un operador. Uno va creando variables ``t1``, ``t2``, ... a medida que las necesita, y la última de ellas es la que almacena el valor final de la fórmula.

Las fórmulas pueden ser arbitrariamente complicadas, como por ejemplo la fórmula

$$
(2-p\,q)\left( \frac{1}{n}+\frac{1}{p+q}\right)
$$

la cual escribimos como

```
(2-p*q)*(1/n+1/(p+q))
```

y cuyo código generado podría ser,

```
t1=p*q
t2=2-t1
t3=1/n
t4=p+q
t5=1/t4
t6=t3+t5
t7=t2*t6
```

El objetivo de esta tarea es que usted implemente una función que, dado un árbol ya construído, lo recorra para generar una secuencia de instrucciones elementales que lo evalúe.

A continuación usted encontrará clases ``Nodoi``, ``Nodoe`` y ``Arbol`` que implementan árboles que representan fórmulas. La clase ``Arbol`` viene con un constructor que recibe un string como parámetro, el cual contiene una fórmula, y este constructor se encarga de transformar la fórmula de string a árbol, de modo que usted no necesita preocuparse de eso. Para simplificar, la fórmula solo podrá contener variables de una sola letra, números de un solo dígito, y sin espacios. Los operadores permitidos son solo suma, resta, multiplicación y división. No hay operador menos unario.

Usted tienen que implementar para la clase ``Arbol`` una función ``codigo`` que al ser invocado genere una secuencia de instrucciones elementales para evaluar la fórmula representada por ese árbol, y la retorne en forma de una lista (de Python).

In [1]:
class Nodoi:
    def __init__(self, izq, info, der):
        self.izq=izq
        self.info=info
        self.der=der
        self.var = ""
    def postorden(self):
        self.izq.postorden()   
        self.der.postorden()
        print(self.info, end=" ")
        
    def variable(self):
        global i
        global codigo
        a = self.izq.variable()
        b = self.der.variable()
        t = 't' + str(i) + '=' + a + self.info + b
        codigo += [t]
        self.var = 't' + str(i)
        i+=1
        return self.var


        
class Nodoe:
    def __init__(self, info):
        self.info=info
    def postorden(self):
        print(self.info, end=" ")
    def variable(self):
        return self.info

class Arbol:
    def __init__(self,formula):
        global k
        global s
        s=formula+";" # agregamos una marca de fin de la entrada
        k=0 # indica próximo caracter por procesar
        # definimos funciones para analizar la fórmula
        def expresion(): # retorna puntero a la raíz de un árbol que representa a la fórmula s
            global k
            global s
            a=factor()
            while s[k]=="+" or s[k]=="-":
                op=s[k]
                k+=1
                b=factor()
                a=Nodoi(a,op,b)
            return a
        
        def factor():
            global k
            global s
            a=termino()
            while s[k]=="*" or s[k]=="/":
                op=s[k]
                k+=1
                b=termino()
                a=Nodoi(a,op,b)
            return a
        
        def termino(): # posible constante, variable o formula parentizada
            global k
            global s
            if s[k].isalpha() or s[k].isdigit():
                a=Nodoe(s[k])
                k+=1
                return a
            if s[k]=="(": # fórmula parentizada
                k+=1
                a=expresion()
                if s[k]!=")":
                    print("Error: Falta cierra paréntesis: "+formula[k:])
                    assert False
                k+=1
                return a
            print("Error: Falta variable, número o abre paréntesis: "+formula[k:])
            assert False
              
        a=expresion()
        if s[k]!=";":
            print("Error: Basura al final de la fórmula: "+formula[k:])
            assert False                
        self.raiz=a
        
    def postorden(self):
        print("Postorden:", end=" ")
        self.raiz.postorden()
        print()
    
    def codigo(self):
        global codigo
        global i
        i = 1
        codigo = []
        if isinstance(self.raiz, Nodoi):
            self.raiz.variable()
        else:
            codigo = ['t' + str(i) + '=' + self.raiz.info]
        return codigo

A continuación probamos esta funcionalidad con varias fórmulas de ejemplo. Para ver si el árbol fue construido correctamente, lo recorremos en postorden para imprimirlo en notación polaca. **Importante**: La notación polaca no tiene nada que ver con lo que ustedes tienen que hacer, es solo una manera de poder imprimir el árbol generado para chequear que esté bien construido.


In [2]:
Arbol("a*b*c").postorden()
Arbol("(a+b)*(c-d)").postorden()
Arbol("(2-p*q)*(1/n+1/(p+q))").postorden()

Postorden: a b * c * 
Postorden: a b + c d - * 
Postorden: 2 p q * - 1 n / 1 p q + / + * 


## Solución

Véase el siguiente ejemplo:
```python
formula = (a+b)*(c+d)
```

La generación de variables para la fórmula viene dado por:
```python
t1 = a+b
t2 = c+d
t3 = t1*t2
```

Nótese que el orden de generación de las variables presenta una estructura similar a un orden postorden, es decir, si se quiere saber la variable asociada a la raíz, se recorre el subárbol izquierdo entero para conocer la variable asociada a éste subarbol y luego se recorre el subárbol derecho entero para conocer la variable asociada a éste subarbol. Finalmente, dada las dos variables, se tiene la variable asociada a la raíz.

Para implementar el procedimiento mencionado anteriormente, primeramente, se modificaron las Clases ``Nodoi`` y ``Nodoe``.

Para la clase ``Nodoi`` se agregó el siguiente atributo:
```python
        self.var = ""
```

Asimismo, se agregó el siguiente método:
```python 
    def variable(self):
        global i
        global codigo
        a = self.izq.variable()
        b = self.der.variable()
        t = 't' + str(i) + '=' + a + self.info + b
        codigo += [t]
        self.var = 't' + str(i)
        i+=1
        return self.var
    ```
El método ``variable`` de la clase ``Ǹodoi`` obtiene la variable asociada al nodo de forma recursiva (obteniendo la variable asociada al hijo izquierdo y obteniendo la variable asociada al hijo derecho). Del mismo modo, en un variable global ``codigo`` guarda la expresion asociada a la variable. La función retorna la variable asociada al nodo.

Para la clase ``Nodoe`` se agregó el siguiente método:
```python 
    def variable(self):
        return self.info
```

Finalmente, dentro de la clase ``Arbol`` se agregó el siguiente método:
```python
    def codigo(self):
        global codigo
        global i
        i = 1
        codigo = []
        if isinstance(self.raiz, Nodoi):
            self.raiz.variable()
        else:
            codigo = ['t' + str(i) + '=' + self.raiz.info]
        return codigo
```
Método que retorna un lista con todas las varibles temporales necesarias para la generación del código.

En caso de tener una fórmula del tipo ``a``, el método retorna:
```python
codigo = ["t1=a"]
```
En caso contrario, recorre recursivamente el árbol para generar las variables necesarias.

## Resultados


En esta sección probamos la generación de código que usted tiene que escribir. Mientras usted no sustituya la función ``codigo`` provisoria que viene en la definición de la clase, va a aparecer la lista que generamos ahí. Primero vamos a definir una función ``probar`` que genera el código a partir de una fórmula dada e imprime el resultado:

In [3]:
def probar(formula):
    a=Arbol(formula)
    for x in a.codigo():
        print(x)

A continuación probamos con varias fórmulas. Además de las fórmulas dadaas, usted tiene que agregar tres ejemplos adicionales.

In [4]:
probar("a+1")

t1=a+1


In [5]:
probar("(a+b)*(c-d)")

t1=a+b
t2=c-d
t3=t1*t2


In [6]:
probar("(2-p*q)*(1/n+1/(p+q))")

t1=p*q
t2=2-t1
t3=1/n
t4=p+q
t5=1/t4
t6=t3+t5
t7=t2*t6


In [7]:
probar("((((a+b)+c)+d)+e)")

t1=a+b
t2=t1+c
t3=t2+d
t4=t3+e


In [8]:
probar("(a+(b+(c+(d+e))))")

t1=d+e
t2=c+t1
t3=b+t2
t4=a+t3


In [9]:
probar("((a+b)+((c+d)*c))+(2-(p+q)+((1/n)+(1/(n+q))))")

t1=a+b
t2=c+d
t3=t2*c
t4=t1+t3
t5=p+q
t6=2-t5
t7=1/n
t8=n+q
t9=1/t8
t10=t7+t9
t11=t6+t10
t12=t4+t11


## Discusión

Suponga que hay escasez de variables temporales, así que se le ha pedido que trate de reciclarlas lo más posible, y solo crear nuevas variables ``ti`` cuando sea estrictamente necesario. Por ejemplo, para la fórmula

```
((a+b)*c)+((d-e)/f)
```

si generamos código como lo hemos hecho en los ejemplos anteriores (código de la izquierda), se usan 5 variables temporales, pero el código de la derecha muestra que se puede evaluar usando solo 2:

```
t1=a+b               t1=a+b
t2=t1+c              t1=t1*c
t3=d-e               t2=d-e
t4=t3/f              t2=t2/f
t5=t2+t4             t1=t1+t2
```

Discuta cómo se podría modificar su algoritmo para que (1) use menos variables temporales, y luego (2) para que use el mínimo posible de variables temporales.

No es necesario que implemente nada, solo que discuta cómo se podría hacer.

(1) Nótese que, inicialmente, la cantidad de variables generadas es igual a la cantidad de nodos internos del árbol. Lo anterior es debido a que, en cada nodo interno, se realiza alguna operación y, dada una operación, se asigna una variable a dicha operación.

De esa forma, para el nodo raíz, si en el hijo izquierdo se utilizaron variables de $1$ a $i$ y en el hijo derecho se utilizaron variables de $i+1$ a $j$, en total en el arbol se ocuparon $j+1$ variables, dado que la expresión para la raíz se define como:
$$ t_{j+1} = t_{i} \circ t_{j} $$

donde $\circ$ es el operador asociado a la raíz. 

Nótese que se puede realizar lo siguiente para la variable de la raíz. Inicialmente se tiene:
$$ t_{j+1} = t_{i} \circ t_{j} $$

Lo cual puede ser escrito de la siguiente forma:
$$ t_{j} = t_{i} \circ t_{j} $$

De esa forma, se reduce en una las variables totales a ocupar.

El procedimiento anterior se puede aplicar a todos los nodos internos del árbol, de esa forma, se reduce el uso de variables temporales.

(2) Ahora, más específicamente, ¿cuántas variables temporales se pueden dejar de usar?. Es decir, dado un árbol que representa una fórmula, ¿de qué forma se pueden utilzar el mínimo de variables temporales para realizar la generación de código de dicho árbol?

Supóngase que se tiene un árbol $A$. Para generar el código, se tiene que recorrer el subárbol izquierdo completo y luego recorrer el subárbol derecho completo. Nótese que, al recorrer el subárbol izquierdo y obtener la variable asociada a éste, se necesita $1$ o $0$ variables para guardar el resultado final (el caso $0$ es cuando el subárbol izquiero es solo una hoja). Por el otro lado, al recorrer el subárbol derecho y obtener la variable asociada a éste, se necesita $1$ o $0$ variables para guardar el resultado final (el caso $0$ es cuando el subárbol derecho es solo una hoja). Con lo anterior, se tienen los siguientes casos:

<ol> 
    <li> El subárbol izquierdo es una hoja y el subárbol derecho también. Luego, para el árbol en total, se necesita utilizar solo $1$ variable temporal. </li>
    <li> El subárbol izquierdo es una hoja y el subárbol derecho es un árbol (nodo interno). Luego, para el árbol en total, se necesitan utilizar $j$ variables temporales, con $j$ la cantidad mínima de variables que se tienen que utilizar para obtener la variable asociada al subárbol derecho </li>
    <li> El subárbol izquierdo es un árbol y el subárbol derecho es una hoja. Luego, para el árbol en total, se necesitan utilizar $i$ variables temporales, con $i$ la cantidad mínima de variables que se tienen que utilizar para obtener la variable asociada al subárbol izquierdo. </li> 
    <li> El subárbol izquierdo es un árbol y el subárbol derecho es un árbol. Ahora, supóngase que el mínimo de variables a utilizar en el árbol izquierdo son $i$. Luego, para el subárbol derecho supóngase que se ocupan como mínimo $j$ nuevas variables. Así, para el árbol en total, se ocupan $ \max \{i, j+1 \}$ variables.</li>
</ol>
   
Con las observaciones anteriores, se puede generar un procedimiento recursivo para utilizar la menor cantidad. Véase el siguiente ejemplo.  
```
    ((a+b)*c)+((d-e)/f)
```

Se tiene el nodo raíz ``+``. Dado el nodo, se recorre el subárbol izquierdo ``((a+b)*c)``. Se repite el procedimiento en el subárbol hasta notar que, el árbol ``(a+b)``  tiene como hijos a dos hojas. Luego, se necesita utilizar 1 variable ``t1`` para almacernar al árbol ``(a+b)``. Ahora, para el árbol ``((a+b)*c)`` necesita solo 1 variable para almacernar el árbol dada las reglas mencionadas anteriormente y, por tanto, se reutiliza la variable ocupada en ``(a+b)``, es decir, se guarda el resultado en un variable ``t1``. Para el árbol,  ``((d-e)/f)`` se repite un procedimiento similar al anterior notando que se necesita solo $1$ variable nueva. Sin embargo, en el subárbol derecho ``((d-e)/f)``, las variables deben empezar en $2$.  Lo anterior es debido a que, como el resultado final del subárbol derecho se debe guardar en una variable, se escoge ``t1`` como la variable y para el subárbol derecho no está disponble esa variable. Luego, para almacenar el subárbol derecho se utiliza ``t2``. Finalmente, se tiene que ``t1 = t1+t2``. Así, el mínimo de variables usadas en el árbol es  $ \max \{1, 1+1\} = 2$ variables. 

Realizando este procedimiento basado en las $4$ reglas expuestas anteriormente, se utiliza el mínimo de variables temporales para realizar la generación de código para cualquier árbol.